In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless')
service = ChromeService(executable_path='C:/Users/Administrator/jupyter notebook/chromedriver/chromedriver.exe')

driver = webdriver.Chrome(service=service, options=options)
driver.get('https://n.news.naver.com/article/055/0001052871?ntype=RANKING')

time.sleep(1)
# find
cbox_chart = driver.find_element(By.CLASS_NAME, 'u_cbox_chart_wrap')
cbox_chart_per = cbox_chart.find_element(By.CLASS_NAME, 'u_cbox_chart_per')

print(cbox_chart_per.text)

time.sleep(10)
driver.close()
# 실습1
# 뉴스의 링크를 넘기면 해당 뉴스의 댓글 통계 정보를 딕셔너리로 반환하는 함수 만들기

def get_stat_gen(url) :
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(2)
    chart_sex = driver.find_element(By.CLASS_NAME, 'u_cbox_chart_sex')
    per_list = chart_sex.find_elements(By.CLASS_NAME, 'u_cbox_chart_per')
    
    dic_keys = ['남자', '여자']
    sex_dic = {}
    
    for key, per in zip(dic_keys, per_list) :
        sex_dic[key] = per.text
    
    return sex_dic

def get_stat_age(url) :

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(2)
    chart_age = driver.find_element(By.CLASS_NAME, 'u_cbox_chart_age')    
    per_list = chart_age.find_elements(By.CLASS_NAME, 'u_cbox_chart_per')
    
    age_keys = ['10대', '20대', '30대', '40대', '50대', '60대']
    age_dic = {}
    
    for key, per in zip(age_keys, per_list) :
        age_dic[key] = per.text
    
    return age_dic
    
def get_reply_stat(url) :
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(2)
    
    ## 댓글 통계 정보 있는지 확인
    try :
        ## u_cbox_chart_wrap를 찾다가 없으면 except쪽 코드가 실행
        driver.find_element(By.CLASS_NAME, 'u_cbox_chart_wrap')         
    except :
        naver_comm = driver.find_element(By.CLASS_NAME, 'is_navercomment')
        url = naver_comm.get_attribute('href')
    
    gen_info = get_stat_gen(url)    
    age_info = get_stat_age(url)
    
    gen_info.update(age_info)
    
    
    return gen_info
   

66%


In [ ]:
# 실습2
# SBS, MBC, KBS 세 언론사의 최근 5일간의 랭킹뉴스 top5의 통계 정보를 json 파일로 저장해주세요.

import my_mod.my_news_util as ut

mbc_url = 'https://media.naver.com/press/214/ranking?type=popular'
kbs_url = 'https://media.naver.com/press/056/ranking?type=popular'
sbs_url = 'https://media.naver.com/press/055/ranking?type=popular'

# ==============================================
# mbc 랭킹뉴스 top5의 통계 정보

def get_top5_news_by_5days(journal_url) :
    dates = ut.get_dates(4)
    news_link = ut.get_news_link_by_time(journal_url, dates)

    my_news_list = []
    for link in news_link :
        page = ut.get_page(link)
        news_list = ut.get_total_news(page)
        my_news_list.extend(news_list[:5])
        
    return my_news_list
    
mbc_news_list = get_top5_news_by_5days(mbc_url)
kbs_news_list = get_top5_news_by_5days(kbs_url)
sbs_news_list = get_top5_news_by_5days(sbs_url)

news_list = []
for news in mbc_news_list :
    try :
        stat_info = get_reply_stat(news['link'])
        news.update(stat_info)
        news_list.append(news)
    except :
        pass
    
stat_list



In [3]:
import my_mod.my_news_util as ut

ut.get_dates(5)

['20230421', '20230422', '20230423', '20230424', '20230425', '20230426']

In [ ]:

# 실습3
# 각 뉴스의 댓글 수를 스크랩해서 저장해주세요.
# 실습4
# 수집한 뉴스데이터를 이용해 다음 문제를 풀어주세요.
## 1. 20대가 가장 많이 본(댓글을 많이 작성한) 뉴스

## 2. 가장 댓글이 적은 뉴스의 댓글 수와 뉴스 링크, 뉴스 번호

## 3. 언론사 번호와 언론사 이름으로 구성된 데이터 프레임을 만들고 merge를 이용해 각 뉴스에 언론사 이름을 붙여주기

## 4. 각 언론사별 평균 댓글 수(댓글 수로 내림차순 정렬)

## 5. 여성의 댓글수가 가장 많은 언론사

## 6. 각 언론사 별 댓글을 많이 작성한 연령대 top3